In [ ]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

##Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

##Libraries

####install

In [ ]:
pip install keras-self-attention

####import

In [ ]:
# For data processing
import numpy as np
import math
from math import sqrt

# For data processing and manipulation
import pandas as pd
import csv

# For date calculations
import datetime
import time

# For ploting data
import IPython
import IPython.display

import itertools
from itertools import cycle
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# For checking path
import os , gc
from os import path
import csv
import json


from scipy.stats import hmean

from sklearn.metrics import mean_squared_error

#tensorflow libs
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping , Callback
import tensorflow as tf
from tensorflow.keras import backend as K
from keras import backend

from tensorflow.keras.layers import *
from tensorflow.keras.layers import Dense , LSTM ,Dropout , PReLU , RepeatVector ,TimeDistributed, Attention
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical , plot_model
from tensorflow.keras import regularizers, constraints, initializers, activations
#from keras.layers.recurrent import Recurrent, _time_distributed_dense
from tensorflow.keras.layers import SimpleRNN as Recurrent
#from tensorflow.compat.v1.keras.layers import RNN 

from tensorflow.keras.layers import InputSpec

from keras_self_attention import SeqSelfAttention
from keras.layers.merge import concatenate


tf.get_logger().setLevel('ERROR')
mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

####Font

In [ ]:
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']

In [ ]:
# The data from 2004 to 2017 are taken
date_after = pd.to_datetime("1/1/2004  00:00:00 AM")
date_before = pd.to_datetime("1/1/2018  00:00:00 AM")

##Rootpath

In [ ]:
ROOTPATH = '/content/gdrive/MyDrive/Projects/Crime forcasting'

##Data Pre-processing

####Load dataset

In [ ]:
#load chicago data
PATH_PDA = ROOTPATH+"/datasets/Chicago/chicago_data_sf_2004_2017_for_police_da.csv"
PATH_DA = ROOTPATH+"/datasets/Chicago/chicago_data_sf_2004_2017_for_da.csv"
PATH_MAIN = ROOTPATH + "/datasets/Chicago/Crimes_-_2001_to_Present.csv"
PATH_WEATHER = ROOTPATH+"/datasets/Chicago/weather_data_chaicago_2004_2017.csv"
PATH_TEMP = ROOTPATH+"/datasets/Chicago/temp_data_chicago_2004_2017.csv"
SF_ATTN_COMMON = ROOTPATH+'/model_commonAttn_mae_swish/attn_LSTM_common_feature'
SF_ATTN_HISTORY = ROOTPATH+'/history_commonAttn_mae_swish/attn_LSTM_common_feature.json'

In [ ]:
dataset = pd.read_csv(PATH_DA)
dataset.head()

In [ ]:
if(dataset['PdDistrict'].isnull().sum()):
    dataset = dataset.drop( index= dataset.loc[dataset['PdDistrict'].isnull()].index[0]) 

In [ ]:
dataset = dataset.reset_index(drop= True)
dataset['datetime'] = dataset['Date']
dataset['PdDistrict'].isnull().sum()

In [ ]:
num_police_dept = dataset['PdDistrict'].nunique()
print("Police departement district = ",num_police_dept)
Police_dept_name = dataset['PdDistrict'].drop_duplicates().sort_values(ascending = True)
print(Police_dept_name)

In [ ]:
# Dictionary for mapping id to name of the unique police department
Police_dept_id_name = {i:Police_dept_name.values[i] for i in range (num_police_dept)}
print("Police Departments ID:Name Dict-> \n",Police_dept_id_name)

# Dictionary for mapping name of the unique police department to id
Police_dept_name_id = {key:value for (value,key) in Police_dept_id_name.items()}
print("Police Departments Name:ID Dict-> \n",Police_dept_name_id)

# Update the new column of dataframe with the value of the list 
dataset["PdDID"] = dataset['PdDistrict']
print("PdDID column -> \n", dataset["PdDID"])

In [ ]:
dataset.to_csv(PATH_PDA,index=False)

In [ ]:
dataset1 = pd.read_csv(PATH_PDA)

In [ ]:
def processCategory(df):
    for col in df.columns:
        col = str(col)
        if(col=='Category'):
            continue
        if col=='datetime':
            continue
        df.drop(col , 1 , inplace=True)

    Category = np.array(df['Category'].unique() , str)
    Category.sort()
    Category = Category.tolist()
    df['Category'] = df['Category'].apply(lambda x:float(Category.index(x)))
    Y = pd.DataFrame(to_categorical(df['Category']))
    df = df.join(Y , on=Y.index)
    df['count'] = df['Category'].apply(lambda x:float(1))
    df.drop('Category' , 1, inplace=True)


    df.set_index("datetime" , inplace = True)
    df.index =  pd.to_datetime(df.index)
    return df, Category

In [ ]:
df2 = processCategory(dataset1)

In [ ]:
df2 , Category2 = df2
df2.head()

In [ ]:
weather_dataset = pd.read_csv(PATH_WEATHER)

In [ ]:
temperature_dataset = pd.DataFrame(weather_dataset[["valid_date_time", "temp"]].copy())
temperature_dataset = temperature_dataset.rename(columns={"valid_date_time": "datetime","temp" : "temperature"})
temperature_dataset["datetime"] = pd.to_datetime(temperature_dataset["datetime"])
temperature_dataset = temperature_dataset.dropna()
temperature_dataset = temperature_dataset[(temperature_dataset.datetime>=date_after) & (temperature_dataset.datetime<date_before)]
temperature_dataset = temperature_dataset.drop_duplicates(subset =["datetime"])
temperature_dataset.shape

In [ ]:
temperature_dataset["temperature"].isnull().sum()

In [ ]:
temperature_dataset.to_csv(PATH_TEMP,index = False)

In [ ]:
def processData(df,sample):
    '''
    Change the sorted crime category with their index value
    & Resample Date time using hour
    input:
        df  : Dataset to process
        sample  : How to resample dataset df
    output:
        df_sample : resampled dataset df
    '''

    df_sample = df.resample(sample).sum()
    return df_sample

In [ ]:
df2 = processData(df2 , '1H')
df2.index = pd.to_datetime(df2.index)
df2 = pd.merge(df2 , temperature_dataset , on='datetime')
df2.set_index('datetime' , inplace=True)
df2.index = pd.to_datetime(df2.index)

In [ ]:
df2.head()

In [ ]:
def dateTimeToSignal(df):
    '''
    Converts the DateTime index to timestamp and convert it to signal (sine and cosine) to deal with periodicity
    input: 
        df : Dataset
    Output:
        df : Dataset with column Day sin , Day cos , Week sin , Week cos ,  Year sin , Year cos ; representing Sin / Cosine signal for timestamp  
    '''
    timestamp_s = df.index.map(datetime.datetime.timestamp)
    day = 24*60*60
    week = 7*day
    year = (365.2425)*day

    df['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
    df['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))

    df['Week sin'] = np.sin(timestamp_s * (2 * np.pi / week))
    df['Week cos'] = np.cos(timestamp_s * (2 * np.pi / week))

    df['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
    df['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
    return df

In [ ]:
df2 = dateTimeToSignal(df2)

In [ ]:
df2.head()

In [ ]:
# Group define
GROUPS = [[15],
     [2],
     [5],
     [8],
     [1],
     [21],
     [3],
     [29,7],
     [26,6],
     [30,22,24,19,27,12,9,16,0,10,14,13,28,18,17,4,23,20,11,25]]

In [ ]:
def make_groups(df):
    '''
    Make 10 groups from 38 different type of crime categories. 
    Where,
      GR0 consits of,
        29 	VEHICLE THEFT 
        7 	DECEPTIVE PRACTICE
      GR1 consits of,
        26 	ROBBERY 
        6 	CRIMINAL TRESPASS
      GR2 consits of,
        30 	WEAPONS VIOLATION 		
        22 	PROSTITUTION 				
        24 	PUBLIC PEACE VIOLATION 		
        19 	OFFENSE INVOLVING CHILDREN 	
        27 	SEX OFFENSE 			
        12 	INTERFERENCE WITH PUBLIC OFFICER 	
        9 	GAMBLING 				
        16 	LIQUOR LAW VIOLATION 			
        0 	ARSON 				  	
        10 	HOMICIDE 				
        14 	KIDNAPPING 				
        13 	INTIMIDATION 				
        28 	STALKING 				
        18 	OBSCENITY 				 
        17 	NON-CRIMINAL 				 
        4 	CONCEALED CARRY LICENSE VIOLATION 	 
        23 	PUBLIC INDECENCY 			 
        20 	OTHER NARCOTIC VIOLATION 		  
        11 	HUMAN TRAFFICKING 			  
        25 	RITUALISM 

    input:
        df : Dataset with count of individual's from 38 categories
    output:
        df : Dataset with count of 10 groups  
    '''
    cnt = 0
    for i in GROUPS:
        cols_to_sum = i
        if(len(i)==1):
            continue
        else:
            newname = "GRP"+str(cnt)
            cnt = cnt + 1
            df[newname] = df[cols_to_sum].sum(axis=1)
            for j in i:
                df.drop(j , 1 , inplace=True)
    return df


In [ ]:
df2 = make_groups(df2) 
df2 = df2.rename(columns={15:1,2: 2, 5 : 3,8 : 4,1 : 5,21 : 6,3 : 7})

In [ ]:
def temp_scale(x):
    '''
    x <= 273K or 0C -> low(0) 
    x > 303K or 30C-> high(2)
    else medium(1)
    '''
    if(x<=0):
        return 0
    elif(x>30):
        return 2
    else:
        return 1

In [ ]:
def split_data(df):
    '''
    split the dataset ( train , val , test = (70 , 20 , 10)% )
    input:
        df : Dataset to split
    output:
        train_df : train dataset
        val_df : validation dataset
        test_df : test dataset
        num_features : Number of features in dataset
        column_indices : column_indices in dataset
    '''
    column_indices = {name: i for i, name in enumerate(df.columns)}

    n = len(df)
    train_df = df[0:int(n*0.7)]
    val_df = df[int(n*0.7):int(n*0.9)]
    test_df = df[int(n*0.9):]

    num_features = df.shape[1]
    return train_df , val_df , test_df , num_features , column_indices

In [ ]:
class WindowGenerator():
    '''
    WindowGenerator Class
    1. Split windows of features into a (features, labels) pairs.
    2. Plot the content of the resulting windows.
    3. Efficiently generate batches of these windows from the training, evaluation, and test data, using tf.data.Datasets S.
    '''
    def __init__(self, input_width, label_width, shift,
               train_df, val_df, test_df,
               label_columns=None , shuffle=False , batch_size = 64):
        '''
        The __init__ method includes all the necessary logic for the input and label indices.
        Input: 
            input_width : input width / window size 
            label_width : output width
            shift : size of window shifting forward
            train_df : train dataset
            val_df : validation dataset
            test_df : test dataset
            label_columns ( Default = None) : Label Columns
            shuffle ( Default = False) : weather to shuffle data 
            batch_size (Default = 64) : Batch Size
        Output: None
        Example :
            w2 = WindowGenerator(input_width=6, label_width=1, shift=1,
                     label_columns=['count'])
            w2

        '''
        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df
        self.shuffle = shuffle
        self.batch_size = batch_size

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in
                                        enumerate(label_columns)}
            
        self.column_indices = {name: i for i, name in
                            enumerate(train_df.columns)}


        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width) #(start , stop)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])
        
    def split_window(self, features):
        '''
        Given a list consecutive inputs, the split_window method will convert them to a window of inputs and a window of labels.
        
        Input: 
            features : Stack of array of datas , used for splitting data to inputs and labels
        Output:
            inputs : nd Array splitted as input_width
            labesl : nd Array splitted as label_width
        Example :
        # Stack three slices, the length of the total window:
            example_window = tf.stack([np.array(train_df[:w2.total_window_size]),
                           np.array(train_df[100:100+w2.total_window_size]),
                           np.array(train_df[200:200+w2.total_window_size])])


            example_inputs, example_labels = w2.split_window(example_window)

            print('All shapes are: (batch, time, features)')
            print(f'Window shape: {example_window.shape}')
            print(f'Inputs shape: {example_inputs.shape}')
            print(f'labels shape: {example_labels.shape}')
        '''
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        #taking only the labels that are presentin the label_columns
        if self.label_columns is not None:
            labels = tf.stack([labels[:, :, self.column_indices[name]] for name in self.label_columns],axis=-1)

        # Slicing doesn't preserve static shape information, so set the shapes
        # manually. This way the `tf.data.Datasets` are easier to inspect.
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels

    def plot(self, model=None, plot_col='count', max_subplots=3):
        '''
            plot method that allows a simple visualization of the split window,
            Input:
                model (Default=None) : tensorflow model to evaluate 
                plot_col ( Default = 'count') : Name of column to evaluate
                max_subplots ( Default = 3) : Maximum Number of subplotting
            Output:
                None
            Example:
                w2.plot()
                w2.plot(plot_col=0) # label wont be shown as w2 config has only column , count 

        '''
        inputs, labels = self.example
        plt.figure(figsize=(12, 8))
        plot_col_index = self.column_indices[plot_col]
        max_n = min(max_subplots, len(inputs))
        for n in range(max_n):
            plt.subplot(max_n, 1, n+1)
            plt.ylabel(f'{plot_col} [normed]')
            plt.plot(self.input_indices, inputs[n, :, plot_col_index],label='Inputs', marker='.', zorder=-10)

            if self.label_columns:
                label_col_index = self.label_columns_indices.get(plot_col, None)
            else:
                label_col_index = plot_col_index

            if label_col_index is None:
                continue

            plt.scatter(self.label_indices, labels[n, :, label_col_index],edgecolors='k', label='Labels', c='#2ca02c', s=64)
            if model is not None:
                predictions = model(inputs)
                plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                        marker='X', edgecolors='k', label='Predictions',c='#ff7f0e', s=64)
            if n == 0:
                plt.legend()
        plt.xlabel('Time [h]')

    def make_dataset(self, data):
        '''
        make_dataset method will take a time series DataFrame and convert it to a
            tf.data.Dataset of (input_window, label_window) pairs using the preprocessing.timeseries_dataset_from_array function.
        Input:
            data :  Input data to transform into (input_window , label_window)
        Output:
            ds : transformed dataset
        '''
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=self.shuffle,
            batch_size=self.batch_size,)
        ds = ds.map(self.split_window)
        return ds
    
    
    def create_dataset2(self , map_df , reshape=True):
      x = []
      y = []
      for res in iter(map_df):
        inputs, labels = res
        if(len(inputs)==64):
          x.append(inputs)
          y.append(labels)
      
      x = np.array(x)
      y = np.array(y)
      if(reshape):
        x = x.reshape(-1, x.shape[-2] , x.shape[-1])
        y = y.reshape(-1 , y.shape[-2] , y.shape[-1])
      return x , y
    
    '''
    properties for accessing  training, validation and test data as tf.data.Datasets using the above make_dataset method.
    Also a standard example batch for easy access and plotting
    '''

    @property
    def train(self):
        return self.make_dataset(self.train_df)

    @property
    def val(self):
        return self.make_dataset(self.val_df)

    @property
    def test(self):
        return self.make_dataset(self.test_df)
    
    
    @property
    def example(self):
        """Get and cache an example batch of `inputs, labels` for plotting."""
        result = getattr(self, '_example', None)
        if result is None:
            # No example batch was found, so get one from the `.train` dataset
            result = next(iter(self.train))
            # And cache it for next time
            self._example = result
        return result

In [ ]:
def create_data(train , test , val , columns):
    '''
    Create dataset from main train , test , val with given columns
    '''
    if(columns==None):
        columns = train.columns
    new_train = train[columns]
    new_test = test[columns]
    new_val = val[columns]
    return new_train , new_test , new_val

In [ ]:
def save_history(history , path):
    # convert the history.history dict to a pandas DataFrame:     
    hist_df = pd.DataFrame(history.history) 
    # or save to csv: 
    hist_csv_file = path
    try:
      with open(hist_csv_file, mode='w') as f:
        hist_df.to_json(f)
    except:
      return None

def get_history(path):
	with open(path) as json_file:
		data = json.load(json_file)
		return data

def save_model_weights(model , path):
  model.save_weights(path)


In [ ]:
def R_squared(y, y_pred):
  residual = tf.reduce_sum(tf.square(tf.subtract(y, y_pred)))
  total = tf.reduce_sum(tf.square(tf.subtract(y, tf.reduce_mean(y))))
  r2 = tf.subtract(1.0, tf.divide(residual, total))
  return r2

In [ ]:
def sMAPE(y, y_pred):
    epsilon = 0.1
    summ = backend.maximum(backend.abs(y) + backend.abs(y_pred) + epsilon, 0.5 + epsilon)
    smape = backend.abs(y_pred - y) / summ * 2.0
    return smape

In [ ]:
plt.rcParams.update({'font.size': 30})
metrics_name = ["MAE" , "MSLE" , "MSE" , "SMAPE" , "R^2"]
def compileModel(model):
    model.compile(loss=tf.losses.MeanAbsoluteError(),optimizer=tf.optimizers.Adam(),
                  metrics=[tf.metrics.MeanSquaredLogarithmicError() , tf.metrics.MeanSquaredError() , sMAPE , R_squared ])
    return model

In [ ]:
MAX_EPOCHS = 200

def compile_and_fit(model, window, name=None , patience=10):
    '''
    Compile and fit a model
    '''
    
    if(name!=None):
        model_path = ROOTPATH+'/model_chicago_common_mae_swish/'+name
        history_path = ROOTPATH+'/history_chicago_common_mae_swish/'+name+".json"
    
        #plot_model(model, model_plot_path, show_shapes=True , expand_nested=True)

    if not path.exists(ROOTPATH+'/model_chicago_common_mae_swish/'):
        os.makedirs(ROOTPATH+'/model_chicago_common_mae_swish/')
        os.makedirs(ROOTPATH+'/history_chicago_common_mae_swish/')

    if(name!=None and path.exists(model_path)):
        print("Loaded Pre Trained Model")
        modelOld = tf.keras.models.load_model(model_path , custom_objects={"sMAPE":sMAPE , "R_squared":R_squared})
        model.set_weights(modelOld.get_weights())
        history = get_history(history_path)
        return model , history

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)
    
    history = model.fit(window.train, epochs=MAX_EPOCHS,validation_data=window.val,callbacks=[early_stopping])
    if(name!=None):
        model.save(model_path)
        save_history(history , history_path)
    return model , history.history

In [ ]:
general_indexs2 = [ 1,2,3,4,5,6,7,'count', 'Day sin' , 'Day cos' , 'Week cos' , 'Week sin', 'Year sin' , 'Year cos' , 'GRP0','GRP1','GRP2']
x_col2 = ['Day sin' , 'Day cos' , 'Year sin' , 'Year cos' , 'Week cos' , 'Week sin' , 'datetime']
def generate_window(df_now):
    train_df , val_df , test_df , num_features_df , column_indices_df = split_data(df_now)
    train_df , test_df , val_df = create_data(train_df , test_df , val_df , general_indexs2)
    
    y_col = []
    for i in train_df.columns:
        if(i in x_col2):
            continue
        y_col.append(i)
    wide_window_all = WindowGenerator(train_df=train_df, test_df=test_df , val_df=val_df, input_width=24, label_width=24, shift=1, label_columns=y_col)
    
    return wide_window_all    

In [ ]:
input_attn_lstm = keras.Input(shape=(24,17), name="input_attn")
x_attn = (keras.layers.Bidirectional(tf.keras.layers.LSTM(128, activation = tf.keras.activations.swish ,return_sequences=True, name= "bdLSTMAttn")))(input_attn_lstm)
x_attn = (SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL, attention_activation='swish', name='Attention'))(x_attn)
# x_attn = (keras.layers.Dense(units=64, activation = tf.keras.activations.swish, name="TimeDisDense1"))(x_attn)
x_attn = (TimeDistributed(keras.layers.Dense(units=64, activation = tf.keras.activations.swish), name="TimeDisDense2"))(x_attn)
output_attn_lstm = (keras.layers.Dense(units=11))(x_attn)
 
attn_lstm_model_common_feature = keras.Model(
    inputs=[input_attn_lstm],
    outputs=[output_attn_lstm],
)
attn_lstm_model_common_feature.summary()


In [ ]:
val_performance_attn = {}
performance_attn = {}
MAX_EPOCHS = 200
wide_window2 = generate_window(df2)

In [ ]:
attn_lstm_model_common_feature = compileModel(attn_lstm_model_common_feature)

attn_lstm_model_common_feature , history_bd  = compile_and_fit(attn_lstm_model_common_feature, window=wide_window2 , name="attn_common_no_transfer")
IPython.display.clear_output()
print("Model is Loaded!!")

val_performance_attn['attn_LSTM_common_feature'] = attn_lstm_model_common_feature.evaluate(wide_window2.val)
performance_attn['attn_LSTM_common_feature'] = attn_lstm_model_common_feature.evaluate(wide_window2.test)
# IPython.display.clear_output()

In [ ]:
print(len(history_bd))

In [ ]:
def transfer_model_load():
    
    if(path.exists(SF_ATTN_COMMON)):
        print("Loaded Pre Trained Model")
        model = tf.keras.models.load_model(SF_ATTN_COMMON)
        history = get_history(SF_ATTN_HISTORY)
        return model , history
    else:
        print("Not found : "+SF_ATTN_COMMON)
        return None , None

In [ ]:
transfer_model, trasnfer_history = transfer_model_load()
IPython.display.clear_output()

In [ ]:
transfer_model = compileModel(transfer_model)
transfer_model , history  = compile_and_fit(transfer_model, wide_window2 , 'attn_common_transfer')

val_performance_attn['chicago_attn_common_transfer'] = transfer_model.evaluate(wide_window2.val)
performance_attn['chicago_attn_common_transfer'] = transfer_model.evaluate(wide_window2.test)

# IPython.display.clear_output()